In [1]:
print("ok")

ok


In [2]:
%pwd

'/Users/pratikshankar/code/genai_projects/genai_medical_diagnosis_chatbot'

In [1]:
## set pwd to root roject dir so that all paths are relative to it and no path issue happens
import os
os.chdir("../")


In [37]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
def load_pdf_file(data):
    loader=DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [5]:
extracted_data=load_pdf_file(data="Data/")

In [6]:
extracted_data[:3]

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data/Medical_book.pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data/Medical_book.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data/Medical_book.pdf', 'total_pages': 637, 'page': 2, 'page_label': '3'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1')]

In [7]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
        length_function=len,
        
    )
    texts = text_splitter.split_documents(extracted_data)
    return texts

In [8]:
text_chunks=text_split(extracted_data=extracted_data)
text_chunks[:3]

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data/Medical_book.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data/Medical_book.pdf', 'total_pages': 637, 'page': 2, 'page_label': '3'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1'),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data/Medical_book.pdf', 'total_pages': 637, 'page': 3, 'page_label': '4'}, page_content='STAFF\nJ

In [9]:
print(len(text_chunks))

5860


In [10]:
text_chunks[5]

Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data/Medical_book.pdf', 'total_pages': 637, 'page': 3, 'page_label': '4'}, page_content='mation presented in this publication, the Gale Group neither guarantees\nthe accuracy of the data contained herein nor assumes any responsibili-\nty for errors, omissions or discrepancies. The Gale Group accepts no\npayment for listing, and inclusion in the publication of any organiza-\ntion, agency, institution, publication, service, or individual does not\nimply endorsement of the editor or publisher. Errors brought to the\nattention of the publisher and verified to the satisfaction of the publish-')

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

/var/folders/k1/x62ylx3s0ql0pz7bqk9qgm0c0000gn/T/ipykernel_11883/3158813755.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [13]:
test_embed=embeddings.embed_query("What is the capital of France?")
print(test_embed)
print(len(test_embed))

[0.08204809576272964, 0.03605550155043602, -0.0038928724825382233, -0.00488103786483407, 0.025651097297668457, -0.057143453508615494, 0.012191587127745152, 0.004678946454077959, 0.03494987264275551, -0.02242191508412361, -0.008005260489881039, -0.10935354977846146, 0.022724755108356476, -0.029320860281586647, -0.04352203384041786, -0.120241180062294, -0.0008486149599775672, -0.018150119110941887, 0.056129567325115204, 0.0030852665659040213, 0.0023363828659057617, -0.016839303076267242, 0.06362467259168625, -0.02366022579371929, 0.03149350732564926, -0.03479793667793274, -0.02054881490767002, -0.00279096863232553, -0.011037997901439667, -0.03612671419978142, 0.05414106324315071, -0.03661713749170303, -0.025008659809827805, -0.03817041590809822, -0.04960361868143082, -0.015148099511861801, 0.021315021440386772, -0.012740444391965866, 0.07670089602470398, 0.044355764985084534, -0.010834885761141777, -0.029759978875517845, -0.01697046123445034, -0.02469186671078205, 0.008087108843028545, 0

In [ ]:
from pinecone import ServerlessSpec
from pinecone import Pinecone
index_name="medical-rag-bot"
PINECONE_API_KEY='

pc=Pinecone(api_key=PINECONE_API_KEY)
from pinecone import ServerlessSpec
if not pc.has_index(name=index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
index=pc.Index(name=index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [22]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [23]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [61]:
#initializing docsearch with the existing index
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [62]:
## initializing retriver object
retriver=docsearch.as_retriever(search_type="similarity",search_kwargs={"k": 3})

In [63]:
## testing on a sample query
query="What is the acne?"
retriver.invoke(input=query)

[Document(id='b9aeca6e-69aa-41f7-87a9-10da9b7932e6', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='2197cd66-1083-4daf-b021-b940418c8bb3', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0

In [29]:
### getting the llm model installing google gen ai
! pip install -q -U google-genai



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GEMINI_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["Explain how AI works"],
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1
    )
)
print(response.text)

Okay, let's break down how AI works, focusing on the core concepts and avoiding overly technical jargon.  Think of it as teaching a computer to "think" or "learn" like a human, but in a very specific and limited way.

**The Basic Idea: Learning from Data**

At its heart, AI is about creating systems that can learn from data, identify patterns, and make decisions or predictions based on those patterns.  Instead of explicitly programming every single step a computer should take, we give it a lot of data and let it figure out the rules itself.

**Key Components and Concepts:**

1.  **Data:** This is the fuel for AI.  It can be anything:
    *   **Images:**  For teaching a computer to recognize objects in pictures.
    *   **Text:**  For understanding language, translating, or writing.
    *   **Numbers:**  For predicting stock prices, analyzing sales trends, or diagnosing medical conditions.
    *   **Audio:** For speech recognition or music generation.
    *   **Video:** For analyzing hu

In [52]:
### now trying gemenai with langchain
! pip install  langchain-google-genai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [66]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    api_key=GEMINI_API_KEY,
    temperature=0.5,
    max_tokens=1024,
    timeout=None,
    max_retries=2
)

In [57]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-6d15926f-d1a7-47ee-abe3-2cbe5eba10c1-0', usage_metadata={'input_tokens': 20, 'output_tokens': 7, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})

In [68]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriver,
    return_source_documents=True
)

query = "What is the main idea of the document?"
result = qa_chain.invoke({"query": query})

print("Answer:", result["result"])
print("\nSources:")

Answer: The document is an excerpt from "The Gale Encyclopedia of Medicine, Second Edition." It includes copyright information and mentions Deepak Chopra's work, particularly his book "Ageless Body, Timeless Mind," where he argues that aging is not an inevitable deterioration and can be influenced and slowed down.

Sources:


In [74]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import Chroma
# from langchain.embeddings import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver, question_answer_chain)

# Query
query = "What is Acromegaly and gigantism?"
response = rag_chain.invoke({"input": query})

print(response["answer"])


Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland, leading to increased bone and soft tissue growth, as well as other bodily disturbances. When this abnormality occurs before bone growth stops, it results in unusual height, known as gigantism. Acromegaly is a relatively rare disorder, affecting approximately 50 out of every one million people.


In [75]:
print("📘 Ask me anything related to medical (type 'exit' to quit):")
while True:
    user_input = input("\n🧠 Your question: ").strip()
    if user_input.lower() in ("exit", "quit"):
        print("👋 Exiting. Have a great day!")
        break

    try:
        response = rag_chain.invoke({"input": user_input})
        print(f"\n🤖 Question: {user_input}")
        print(f"\n🤖 Answer: {response['answer']}")
    except Exception as e:
        print(f"⚠️ Error: {e}")

📘 Ask me anything related to medical (type 'exit' to quit):

🤖 Question: hello

🤖 Answer: Hello! How can I help you?

🤖 Question: my asshole hurts

🤖 Answer: There are several potential causes of pain in the anorectal area, including anal fissures, anorectal abscesses, and anorectal fistulas. Anal fissures are tears in the lining of the anus, often caused by hard bowel movements. Anorectal abscesses are pus-forming infections, while anorectal fistulas are abnormal openings from the anorectal area to another part of the body.
👋 Exiting. Have a great day!
